In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
def show(img):
    cv2.imshow("source",img)
    cv2.waitKey(0)

In [2]:
img = cv2.resize(cv2.imread("pic\\1225.jpg"),(384,288))
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [3]:
# Image Enhancement
base_vertex = [(x,y) for x in range(0,9) for y in range(0,9)]
pos_vertex = map(lambda x:(x[0]*36,x[1]*48),base_vertex)
vermap = lambda x,y:x*8+y
IWmean = [ img[
    0   if i[0] == 0 else i[0] - 18:
    288 if i[0] == 288 else i[0] + 18,
    0   if i[1] == 0 else i[1] - 24:
    384 if i[1] == 384 else i[1] + 24]
             .mean() for i in pos_vertex]
IWstd = [ img[
    0   if i[0] == 0 else i[0] - 18:
    288 if i[0] == 288 else i[0] + 18,
    0   if i[1] == 0 else i[1] - 24:
    384 if i[1] == 384 else i[1] + 24]
             .std() for i in pos_vertex]
# Cx = (j-48n) * 1.0/48
# Cy = (i-36m) * 1.0/36
# Imean(i,j) = (1-Cy) * ((1 - Cx) * IWmean(A) + CxIWmean(B))
#              + Cy * ((1 - Cx) * IWmean(C) + CxIWmean(D))
# Istd(i,j)  = (1-Cy) * ((1 - Cx) * IWstd(A) + CxIWstd(B))
#              + Cy * ((1 - Cx) * IWstd(C) + CxIWstd(D))
def getImean(i,j):
    m = i//36
    n = j//48
    Cx = (j-48*n) * 1.0/48
    Cy = (i-36*m) * 1.0/36
    Imean = (1-Cy) * ((1 - Cx) * IWmean[vermap(m,n)]   + Cx*IWmean[vermap(m,n+1)])\
              + Cy * ((1 - Cx) * IWmean[vermap(m+1,n)] + Cx*IWmean[vermap(m+1,n+1)])
    Istd  = (1-Cy) * ((1 - Cx) * IWstd[vermap(m,n)]    + Cx*IWstd[vermap(m,n+1)])\
              + Cy * ((1 - Cx) * IWstd[vermap(m+1,n)]  + Cx*IWstd[vermap(m+1,n+1)])
    return Imean,Istd 
def getCoe(std):
    if std < 20:
        return 3.0 / ((2.0 / 400) * (std - 20) *(std - 20) + 1)
    elif std < 60:
        return 3.0 / ((2.0 / 1600) * (std - 20) *(std - 20) + 1)
    else:
        return 1        
def doImap(img):
    ret = np.empty_like(img)
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
            val = getImean(i,j)
            coe = getCoe(val[1])
            ret[i,j] = coe * (img[i,j] - val[0]) + val[0]
    return ret
img2 = doImap(img)

In [4]:
img2 = cv2.convertScaleAbs(cv2.Sobel(img2,cv2.CV_64F,2,0,11))
tmp,img3 = cv2.threshold(img2,75,255,cv2.THRESH_BINARY)

In [27]:
# Curve remove
M = np.zeros_like(img)
N = np.zeros_like(img)
Tlong = 100
Tshort = 0
for i in range(2,288):
    for j in range(2,384):
        if img3[i, j] == 255:
            if img3[i-1, j-1] + img3[i-1, j] + img3[i-1, j+1] + img3[i, j-1] > 0:
                M[i, j] = max(M[i-1, j-1], M[i-1, j], M[i-1, j+1], M[i, j-1]) + 1
            else:
                M[i, j] = max(M[i-2, j-1], M[i-2, j], M[i-2, j+1],
                             M[i-1, j-2], M[i-1, j+2], M[i, j-2]) + 1
for i in range(0,286)[::-1]:
    for j in range(0,382)[::-1]:
        if img3[i, j] == 255:
            if img3[i+1,j-1] + img3[i+1, j] + img3[i+1, j+1] + img3[i, j+1] > 0:
                N[i, j] = max(N[i+1, j-1], N[i+1, j], N[i+1, j+1], N[i, j+1]) + 1
            else:
                N[i, j] = max(N[i+2, j-1], N[i+2, j], N[i+2, j+1],
                             N[i+1, j-2], N[i+1, j+2], N[i, j+2]) + 1
for i in range(0,288):
    for j in range(0,384):
        if img3[i,j] == 255:
            if M[i,j] + N[i,j] >Tlong or M[i,j] + N[i,j] < Tshort:
                img3[i,j] = 0

In [28]:
show(img3)

In [5]:
def cluster(img):
    kernelX = cv2.getStructuringElement(cv2.MORPH_RECT,(12, 1))  
    closed = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernelX)
    return cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernelX)
def antiNoise(img):
    kernelX = cv2.getStructuringElement(cv2.MORPH_RECT,(1, 6))  
    closed = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernelX)
    return cv2.morphologyEx(closed, cv2.MORPH_CLOSE, kernelX)

img4 = cluster(antiNoise(img3))
img5 = cv2.erode(img4, None, iterations = 4)
img5 = cv2.dilate(img5, None, iterations = 4)

contours, hierarchy,_ = cv2.findContours(img5.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

In [34]:
# rect find
img0 = img.copy()
rect = map(cv2.minAreaRect,hierarchy)
rect = map(cv2.boxPoints,rect)
rect = map(cv2.boundingRect,rect)

In [35]:
rect

[(378, 188, 5, 33),
 (16, 162, 34, 15),
 (226, 160, 90, 25),
 (72, 123, 28, 18),
 (116, 122, 41, 48),
 (355, 114, 21, 20),
 (1, 98, 33, 35),
 (100, 88, 12, 12),
 (49, 75, 16, 14),
 (341, 57, 30, 15),
 (229, 42, 44, 23),
 (332, 35, 21, 16),
 (356, 14, 12, 9),
 (1, 1, 170, 79)]

In [36]:
#rect filter
def squareFilter(x):
    area = x[2] * x[3]
    return area <= 288 * 384 * 0.05 and area >= 288 * 384 * 0.01
def heightFilter(x):
    return x[1] <= 288 * 0.9 and x[1] >= 288 * 0.2
def hwRateFilter(x):
    return x[2] > x[3] * 1.5  and x[2] < x[3] * 8.8
rect = filter(squareFilter,rect)
rect = filter(heightFilter,rect)
rect = filter(hwRateFilter,rect)

In [37]:
# rect draw
def draw_rect(x):
    cv2.rectangle(img0,(x[0],x[1]),(x[0]+x[2],x[1]+x[3]),(0,255,255))
map(draw_rect,rect)
show(img0)

In [2]:
label_file = open('detectlabel.txt')
label = label_file.readlines()

In [5]:
label = label[:10]

In [21]:
location = map(lambda x:map(lambda x:filter(lambda x:x!='',x.split(' ')),x[1:]),label1)

In [27]:
[y for y in x for x in location]

[['2029', '739', '2040', '924', '1472', '1212', '1447', '982'],
 ['2029', '739', '2040', '924', '1472', '1212', '1447', '982'],
 ['2029', '739', '2040', '924', '1472', '1212', '1447', '982'],
 ['2029', '739', '2040', '924', '1472', '1212', '1447', '982'],
 ['2029', '739', '2040', '924', '1472', '1212', '1447', '982'],
 ['2029', '739', '2040', '924', '1472', '1212', '1447', '982'],
 ['2029', '739', '2040', '924', '1472', '1212', '1447', '982'],
 ['2029', '739', '2040', '924', '1472', '1212', '1447', '982'],
 ['2029', '739', '2040', '924', '1472', '1212', '1447', '982'],
 ['2029', '739', '2040', '924', '1472', '1212', '1447', '982']]